# Axis estimation by circle fitting

Ankle movements approximation and noise added circle regression.

In [1]:
#Model initialization
from sage.plot.plot3d.shapes2 import *

# Rotation axis in a unitary vector and angle in degrees
def rot_ax_ang(u,a):
    #degrees to radians
    alpha=a*pi/180
    na=(1-cos(alpha))
    ca=cos(alpha)
    sa=sin(alpha)
    #rotation matrix
    R=matrix([[u[0]^2*na+ca, u[0]*u[1]*na-u[2]*sa, u[0]*u[2]*na+u[1]*sa],
            [u[0]*u[1]*na+u[2]*sa, u[1]^2*na+ca, u[1]*u[2]*na-u[0]*sa],
            [u[2]*u[0]*na-u[1]*sa, u[2]*u[1]*na+u[0]*sa, u[2]^2*na+ca]])
    return R

pvar=0.9 #percentual variation
#Talocrural direction
TCa1=-6*pvar
TCa2=80*pvar
vTC=rot_ax_ang([0,0,1], TCa1)*rot_ax_ang([1,0,0], TCa2)*vector([0,0,1])
#Subtalar direction
STa1=23*pvar
STa2=41*pvar
vST=rot_ax_ang([0,0,1], STa1)*rot_ax_ang([0,1,0], STa2)*vector([1,0,0])

#Anthropometric values
#vK=12;vL=11;vO=16;vP=1;vQ=5;vR=0.54 #anthropometric measurements
vK=12*pvar;vL=11*pvar;vO=16*pvar;vP=1*pvar;vQ=5*pvar;vR=0.54*pvar #anthropometric measurements
#H=1750 #heigth in mm
H=1750*pvar #heigth in mm
d_m=(0.285-0.039)*H/6 #knee-ankle half distance
d_p=0.039*H #ankle-foot
z_p=-(d_m+d_p) # z initial position 
r_p=0.055*H*2/3 #platform radius 
ae=2*pi/3 #auxiliar angle
PO=vector([0,0,0]) #Origin at middle shank

#initial platform position
A0=vector([r_p,0,z_p]) 
B0=vector([r_p*cos(ae),r_p*sin(ae),z_p])
C0=vector([r_p*cos(-ae),r_p*sin(-ae),z_p])
PM0=(A0+B0+C0)/3 #center of the platform

#marker representation
dot=point3d((0,0,0),size=5,color='red',opacity=.5)
dPO=dot.translate(PO)
dA0=dot.translate(A0)
dB0=dot.translate(B0)
dC0=dot.translate(C0)
dPM0=dot.translate(PM0)

r_1=vector([0,0,-d_m]) #intersection point between the talocrural axis and the sagittal plane
ap=0.039*H 
lp=0.152*H

#points on the malleolar medial and lateral
M_1=r_1-ap*vR*vTC
M_2=r_1+ap*(1-vR)*vTC
tht=(STa2*pi/180)

#subtalar axis and the sagittal plane intersection 
r_2=vQ*vector([-cos(tht),0,-sin(tht)])+r_1

#points from the hindfoot to the midfoot
N_1=r_2+0.6*lp*vR*vST
N_2=r_2-0.3*lp*vR*vST

#representation of the kinematic chain
lrt=line([PO,r_1])
lm12=line([M_1,M_2])
ln12=line([N_1,N_2])
lrA=line([r_2,A0])
lrB=line([r_2,B0])
lrC=line([r_2,C0])

#markers representation
dr1=dot.translate(r_1)
dr2=dot.translate(r_2)
dM1=dot.translate(M_1)
dM2=dot.translate(M_2)
dN1=dot.translate(N_1)
dN2=dot.translate(N_2)

#reference labels
plbl=vector([0,0,8])
lbp = text3d('PO',PO+plbl)
lbp += Text('A0',color='black').translate(A0++plbl)
lbp += text3d('B0',B0++plbl)
lbp += text3d('C0',C0++plbl)
lbp += text3d('PM0',PM0++plbl)
#axes labels
lbf = text3d('M1',M_1++plbl)
lbf += text3d('M2',M_2++plbl)
lbf += text3d('N1',N_1++plbl)
lbf += text3d('N2',N_2++plbl)
lbf += text3d('r1',r_1++plbl)
lbf += text3d('r2',r_2++plbl)


#finding the twist unitary vectors 
w1=n((M_1-M_2)/abs(M_1-M_2))
w2=n((N_1-N_2)/abs(N_1-N_2))

#the perpendicular component
v1=n(-w1.cross_product(r_1))
v2=n(-w2.cross_product(r_2))

#angles for the talocrural and subtalar rotations
theta1=var('theta_1')
theta2=var('theta_2')

#six dimensional vector xi mapping 
xi1=matrix([v1[0],v1[1],v1[2],w1[0],w1[1],w1[2]]).transpose()
xi2=matrix([v2[0],v2[1],v2[2],w2[0],w2[1],w2[2]]).transpose()

#transformation of exponential matrix of rotation
Rexp1=rot_ax_ang(w1,theta1)

#rotation matrix component of the homogeneous transformation
vexp1=(matrix.identity(3)-Rexp1)*(w1.cross_product(v1))

#conformation of the homogeneous transformation matrix
MTH1=(Rexp1.augment(vexp1)).stack(vector([0,0,0,1]))

#components for the subtalar axis
Rexp2=rot_ax_ang(w2,theta2)
vexp2=(matrix.identity(3)-Rexp2)*(w2.cross_product(v2))
MTH2=(Rexp2.augment(vexp2)).stack(vector([0,0,0,1]))

#transformation matrix representing the initial position
gst0A=matrix([[1,0,0,A0[0]],[0,1,0,A0[1]],[0,0,1,A0[2]],[0,0,0,1]])
gst0B=matrix([[1,0,0,B0[0]],[0,1,0,B0[1]],[0,0,1,B0[2]],[0,0,0,1]])
gst0C=matrix([[1,0,0,C0[0]],[0,1,0,C0[1]],[0,0,1,C0[2]],[0,0,0,1]])
gst0PM=matrix([[1,0,0,PM0[0]],[0,1,0,PM0[1]],[0,0,1,PM0[2]],[0,0,0,1]])

#product of exponential matrices for all the points
MTHA=MTH1*MTH2*gst0A
MTHB=MTH1*MTH2*gst0B
MTHC=MTH1*MTH2*gst0C
MTHPM=MTH1*MTH2*gst0PM

#components of the group of rigid movements for the central point
f_xpm=MTHPM[0][3]
f_ypm=MTHPM[1][3]
f_zpm=MTHPM[2][3]
#orthogonal direction vectors
f_spm=vector([MTHPM[0][0],MTHPM[1][0],MTHPM[2][0]])
f_npm=vector([MTHPM[0][1],MTHPM[1][1],MTHPM[2][1]])
f_apm=vector([MTHPM[0][2],MTHPM[1][2],MTHPM[2][2]])

#components of the three vertices of the plattform
f_xA=MTHA[0][3]
f_yA=MTHA[1][3]
f_zA=MTHA[2][3]
f_xB=MTHB[0][3]
f_yB=MTHB[1][3]
f_zB=MTHB[2][3]
f_xC=MTHC[0][3]
f_yC=MTHC[1][3]
f_zC=MTHC[2][3]

sc=30 #scale factor
ar1=arrow3d(PO,r_1,sc,color='cyan')
ar2=arrow3d(PO,r_2,sc,color='magenta')
aw1=arrow3d(r_1, r_1 + sc*w1,sc, color ='black')
aw2=arrow3d(r_2, r_2 + sc*w2,sc, color = 'black')
av1=arrow3d(r_1, r_1 + sc*v1/abs(r_1),sc, color ='black')
av2=arrow3d(r_2, r_2 + sc*v2/abs(r_2),sc, color ='black')
#orientation frame at PM0    
spm=vector([f_spm[0].subs(theta1==0,theta2==0),
            f_spm[1].subs(theta1==0,theta2==0),
            f_spm[2].subs(theta1==0,theta2==0)]).n()
npm=vector([f_npm[0].subs(theta1==0,theta2==0),
           f_npm[1].subs(theta1==0,theta2==0),
           f_npm[2].subs(theta1==0,theta2==0)]).n()
apm=vector([f_apm[0].subs(theta1==0,theta2==0),
           f_apm[1].subs(theta1==0,theta2==0),
           f_apm[2].subs(theta1==0,theta2==0)])
spmv=arrow3d(PM0, PM0 + sc*spm,sc, color ='red')
npmv=arrow3d(PM0, PM0 + sc*npm,sc, color ='green')
apmv=arrow3d(PM0, PM0 + sc*apm,sc, color ='blue')

scrws = ar1 +ar2+ aw1+aw2 + av1 + av2 + spmv + npmv + apmv

#trajectory holding constant angles
tmin=-20
tmax=20
dp=50
t2kpm=MTHPM.subs(theta2==0)
trkt2=parametric_plot3d([t2kpm[0][3],t2kpm[1][3],t2kpm[2][3]],
                        (theta_1,tmin,tmax),
                        texture= 'red', thickness='3', plot_points=dp)
t1kpm=MTHPM.subs(theta1==0)
trkt1=parametric_plot3d([t1kpm[0][3],t1kpm[1][3],t1kpm[2][3]],
                        (theta_2,tmin,tmax),
                        texture= 'green', thickness='3', plot_points=dp)

#trajectory holding constant angles
tmin=-20
tmax=20
dp=50
t2ka=MTHA.subs(theta2==0)
trkt2a=parametric_plot3d([t2ka[0][3],t2ka[1][3],t2ka[2][3]],
                        (theta_1,tmin,tmax),
                        texture= 'red', thickness='3', plot_points=dp)
t1ka=MTHA.subs(theta1==0)
trkt1a=parametric_plot3d([t1ka[0][3],t1ka[1][3],t1ka[2][3]],
                        (theta_2,tmin,tmax),
                        texture= 'green', thickness='3', plot_points=dp)
#trajectory holding constant angles
tmin=-20
tmax=20
dp=50
t2kb=MTHB.subs(theta2==0)
trkt2b=parametric_plot3d([t2kb[0][3],t2kb[1][3],t2kb[2][3]],
                        (theta_1,tmin,tmax),
                        texture= 'red', thickness='3', plot_points=dp)
t1kb=MTHB.subs(theta1==0)
trkt1b=parametric_plot3d([t1kb[0][3],t1kb[1][3],t1kb[2][3]],
                        (theta_2,tmin,tmax),
                        texture= 'green', thickness='3', plot_points=dp)
#trajectory holding constant angles
tmin=-20
tmax=20
dp=50
t2kc=MTHC.subs(theta2==0)
trkt2c=parametric_plot3d([t2kc[0][3],t2kc[1][3],t2kc[2][3]],
                        (theta_1,tmin,tmax),
                        texture= 'red', thickness='3', plot_points=dp)
t1kc=MTHC.subs(theta1==0)
trkt1c=parametric_plot3d([t1kc[0][3],t1kc[1][3],t1kc[2][3]],
                        (theta_2,tmin,tmax),
                        texture= 'green', thickness='3', plot_points=dp)



#normal plane to TC axis
tcpl = implicit_plot3d(lambda x,y,z: w1.dot_product(vector([x,y,z])-PM0), 
                       (-20+PM0[0],20+PM0[0]), (-20+PM0[1],20+PM0[1]), 
                       (-20+PM0[2],20+PM0[2]),color='green',opacity=0.5)
#normal plane to ST axis
stpl = implicit_plot3d(lambda x,y,z: w2.dot_product(vector([x,y,z])-PM0), 
                       (-20+PM0[0],20+PM0[0]), (-20+PM0[1],20+PM0[1]), 
                       (-20+PM0[2],20+PM0[2]), color ='red',opacity=0.5)

#trajectory angles depending on parameter t
dp=100
tmax=180
var('t')
tpm=MTHPM.subs(theta1==t,theta2==t)
vectpm = vector([tpm[0][3],tpm[1][3],tpm[2][3]])
ttpm=parametric_plot3d(vectpm, (t,0,tmax),
                        texture= 'blue', thickness='3', plot_points=dp)
#mapped trajectory
prjtr=vectpm - vectpm.dot_product(apm)*(apm)
pltrj=parametric_plot3d(prjtr  + PM0, (t,0,tmax),
                        texture= 'cyan', thickness='3', plot_points=dp)
#mapped trajectory to w1
prjw1=prjtr - prjtr.dot_product(w1)*(w1)
pltw1=parametric_plot3d(prjw1  + PM0, (t,0,tmax),
                        texture= 'yellow', thickness='3', plot_points=dp)

#mapped trajectory to w2
prjw2=prjtr - prjtr.dot_product(w2)*(w2)
pltw2=parametric_plot3d(prjw2  + PM0, (t,0,tmax),
                        texture= 'yellow', thickness='3', plot_points=dp)
#platform initial position
pltf= polygon([A0,B0,C0],color='gray',opacity=0.5)

#show the initial position
gr = dPO+dA0+dB0+dC0+dPM0 #+ trkt2 + trkt1 + tcpl + stpl + ttpm + pltrj + pltw1 + pltw2
gr+= lrt+lm12+ln12+lrA+lrB+lrC
gr+= dM1+dM2+dr1+dr2+dN1+dN2


gr+= lbp+lbf+scrws+pltf+lbp
gr.show(viewer='jmol', antialiasing=True, raydepth=3,
    figsize=[9,6], # the image resolution is 100*figsize
    camera_position=[3, 4, 2], # a distant camera position combined with 
    zoom=1, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.0,-3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=1)
 

Graphics3d Object

Noisy trajectories generation

In [2]:
   
#get the points dictionary trajectory theta_2 constant, TC axis
pdic=trkt2a.threejs_repr(trkt2a.default_render_params())[0][1]
#pdic=trkt2b.threejs_repr(trkt2b.default_render_params())[0][1]
#pdic=trkt2c.threejs_repr(trkt2c.default_render_params())[0][1]
#pdic=trkt2.threejs_repr(trkt2.default_render_params())[0][1]

#get the points dictionary trajectory theta_1 constant, ST axis
#pdic=trkt1a.threejs_repr(trkt1a.default_render_params())[0][1]
#pdic=trkt1b.threejs_repr(trkt1b.default_render_params())[0][1]
#pdic=trkt1c.threejs_repr(trkt1c.default_render_params())[0][1]
#pdic=trkt1.threejs_repr(trkt1.default_render_params())[0][1]

#get the point list
plist=pdic[list(pdic)[0]]
#point matrix
pmatrix=matrix(plist)
#list plot
list_plot3d([pmatrix])
#noise matrix
mnoise = random_matrix(QQ, len(plist), 3, num_bound=20, den_bound=10)/10
#add noise to pmatrix
npmatrix=mnoise+pmatrix

Circle fitting

In [3]:
#findind a, b and d
mA=matrix([pmatrix[:,0].list(),pmatrix[:,1].list(),ones_matrix(len(plist),1).list()]).T
vB=-vector(pmatrix[:,2].list())
psA=mA.pseudoinverse()
sabd=psA*vB
aval=sabd[0]
bval=sabd[1]
dval=sabd[2]

#finding c
import numpy
xm=numpy.mean([pmatrix[:,0].list()])
ym=numpy.mean([pmatrix[:,1].list()])
zm=numpy.mean([pmatrix[:,2].list()])
cval=-(xm*aval+ym*bval+dval)/zm

#unitary normal vector to plane
npl = vector([aval, bval, cval]) 
nsu=npl/norm(npl)
nsu

(-0.08950232461911863, -0.9468356938113643, 0.3090169943749471)

In [4]:
#angle with XY plane
vz=vector([0,0,1])
arot=acos(nsu.dot_product(vz))
rvec=nsu.cross_product(vz)

Prot=[vector(pmatrix[i])*cos(arot)+ (rvec.cross_product(vector(pmatrix[i])))*sin(arot) + 
      rvec*(rvec.dot_product(vector(pmatrix[i])))*(1-cos(arot))for i in range(len(plist))]
protmed=median(Prot)

var('xc,yc,rc')
var('x,y')
prmat=matrix(Prot)
mA=matrix([prmat[:,0].list(),prmat[:,1].list(),ones_matrix(len(plist),1).list()]).T
vB=vector([prmat[i,0]^2+prmat[i,1]^2for i in range(len(plist))])
psA=mA.pseudoinverse()
cfit=psA*vB
mdl=(x-xc)^2+(y-yc)^2-rc^2
mdl=mdl.simplify_full()
var('c0,c1,c2')
cvec=vector([c0,c1,c2])
minfun=(vB-mA*cvec).dot_product(vB-mA*cvec)

data=matrix(Prot)
data.delete_columns([2])
model(x,y) = mdl
fitop=find_fit(data,model)
rcop=fitop[0].rhs()
xcop=fitop[1].rhs()
ycop=fitop[2].rhs()
smin=minimize(minfun,cfit)
eqc1=2*xc==smin[0]
eqc2=2*yc==smin[1]
eqc3=rc^2-xc^2-yc^2==smin[2]
slnc=solve([eqc1,eqc2,eqc3],[rc,xc,yc])
tcr=slnc[0][0].rhs().n()
tcxc=slnc[0][1].rhs().n()
tcyc=slnc[0][2].rhs().n()
cprt=vector([xcop,ycop,protmed[2]])
protmed=median(Prot)
uvpr=(protmed-cprt)/norm(protmed-cprt)
vvpr=vz.cross_product(uvpr)
cirpar=cprt+rcop*(uvpr*cos(t) + vvpr*sin(t))

from sage.plot.arc import Arc
pmedian=median(pmatrix)
pmed3d=point3d(pmedian,size=5,color='blue')
crot=vector([xcop,ycop,protmed[2]])
crot3d=point3d(crot,size=5,color='blue')
cptc=rot_ax_ang(rvec,-arot*180/pi)*crot
croti=crot*cos(-arot)+ (rvec.cross_product(crot))*sin(-arot) + rvec*(rvec.dot_product(crot*(1-cos(-arot))))
norm(r_1-r_2).n()-abs(norm(protmed-cprt)-norm(pmedian-croti))

prpt=point3d(Prot)
w1arw=arrow3d(croti, croti - sc*nsu,sc, color ='green')
r1arw=arrow3d(PO, croti ,sc, color ='green')
r2arw=arrow3d(PO, crot ,sc, color ='green')
r3arw=arrow3d(PO, pmedian ,sc, color ='green')
r4arw=arrow3d(PO, protmed ,sc, color ='green')
plcir=parametric_plot3d(cirpar,(t,-pi,pi))
plpmat=point3d(pmatrix,size=3,color='red')
plctr=point3d(cptc,size=3,color='green')
plt=plctr+plpmat+w1arw+pmed3d+plcir+prpt+crot3d+r1arw+r2arw+r3arw+r4arw+gr
plt.show(viewer='threejs', antialiasing=True, raydepth=3,
    figsize=[9,6], # the image resolution is 100*figsize
    camera_position=[3, 4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.0,-3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=1)
plt.save('fitcira.png',viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, 4, 2], # a distant camera position combined with 
    zoom=2.5, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,-3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=1)

Graphics3d Object

In [5]:
#rotation center
croti.n(digits=4)
#Talocrural direction
nsu.n(digits=2)
#radius
rcop.n(digits=4)

# line median center for TC axis
cntrm=matrix([[44.44, 18.25, -90.08],[17.57, 6.768, -69.25],[1.578, 1.819, -58.07],[20.87, 8.882, -72.81]])

# line median center for ST axis
#cntrm=matrix([[0.8649, 21.38, -67.12],[5.713, 55.31, -78.24],[-2.442, -26.69, -53.15],[1.552, 16.42, -66.83]])

cr_m=median(cntrm).n(digits=3)

#median line normal vector TC axis
pnvec=matrix([[-0.75, -0.28, 0.60],[-0.75, -0.28, 0.60],[-0.75, -0.28, 0.60],[-0.75, -0.28, 0.60]])

#median line normal vector ST axis
#pnvec=matrix([[-0.089, -0.95, 0.31],[-0.089, -0.95, 0.31],[-0.089, -0.95, 0.31],[-0.089, -0.95, 0.31]])

pn_m=median(pnvec).n(digits=3)

#Sagittal plane y=0 normal vector
ny0=vector([0,1,0])

#find d
d=(-cr_m.dot_product(ny0))/(pn_m.dot_product(ny0))

#replace in line equation r1
r_1i=cr_m+pn_m*d

#replace in line equation r2
#r_2i=cr_m+pn_m*d

#show r_1 and omega_1
print('r_1=',r_1i.n(digits=3),'omega_1=',pn_m)

#show r_2 and omega_2
#print('r_2=',r_2i.n(digits=3),'omega_2=',pn_m)

#in plucker coordinates
m_1=r_1i.cross_product(pn_m).n(digits=3)

#TC axis plucker coordinates
print('Plucker1:',pn_m.list()+m_1.list())

#ST axis plucker coordinates
#print('Plucker2:',pn_m.list()+m_1.list())

#the P0 point is the median
print('P_0=',pmedian.n(digits=3))
print('median center:',cr_m,'median normal:',pn_m)

r_1= (-1.74, 0.000, -54.3) omega_1= (-0.750, -0.280, 0.600)
Plucker1: [-0.750, -0.280, 0.600, -15.2, 41.7, 0.487]
P_0= (57.8, 0.000580, -126.)
median center: (19.2, 7.83, -71.0) median normal: (-0.750, -0.280, 0.600)
